In [1]:
from qiskit.transpiler.passes.synthesis import UnitarySynthesis
from qiskit.transpiler.passes import TemplateOptimization
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import PassManager
from qiskit.visualization import circuit_drawer
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator

In [ ]:
qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)
qc.cx(0,1)
qc.h(0)
qc.h(1)

# pass_manager = PassManager(TemplateOptimization())
# optimized_qc = pass_manager.run(qc)

optimized_qc = transpile(qc, optimization_level=3)

print("Optimized Circuit:")
print(optimized_qc.draw())

Optimized Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├
     ├───┤┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘


In [ ]:
from qiskit import QuantumCircuit, transpile

# Original circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)
qc.cx(0, 1)
qc.h(0)
qc.h(1)

# Transpile with optimization
optimized_qc = transpile(qc, optimization_level=3) # basis_gates=['h', 'x', 'cx', 'cz', 'ry', 'u', 'u1', 'u2']

# View the result
print(qc.draw())
print(optimized_qc.draw())

     ┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤ H ├
     ├───┤└─┬─┘├───┤
q_1: ┤ H ├──■──┤ H ├
     └───┘     └───┘
     ┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤ H ├
     ├───┤└─┬─┘├───┤
q_1: ┤ H ├──■──┤ H ├
     └───┘     └───┘


In [25]:
unitary = Operator(qc).data
unitary

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]])

In [33]:
# unitary synthesis
qc = QuantumCircuit(2)
qc.unitary(unitary, range(2))
synth_pass = UnitarySynthesis(basis_gates=['h', 'x', 'cx', 'cz', 'ry', 'u', 'u1', 'u2'])
pm = PassManager(synth_pass)
optimized_circuit = pm.run(qc)

print(qc.draw())
print(optimized_circuit.draw())

     ┌──────────┐
q_0: ┤0         ├
     │  Unitary │
q_1: ┤1         ├
     └──────────┘
global phase: 0
     ┌───────────────────┐     ┌───────────────────┐
q_0: ┤ U(π/2,-0.32041,0) ├──■──┤ U(π/2,-π,0.32041) ├
     ├───────────────────┤┌─┴─┐├───────────────────┤
q_1: ┤ U(π/2,-π,0.95425) ├┤ X ├┤ U(π/2,-0.95425,0) ├
     └───────────────────┘└───┘└───────────────────┘


In [35]:
from qiskit.transpiler.passes import Optimize1qGates, CommutativeCancellation, Collect2qBlocks, UnitarySynthesis, RemoveDiagonalGatesBeforeMeasure, OptimizeSwapBeforeMeasure, TemplateOptimization

def qiskit_transpiler(qc):
    # backend = Aer.get_backend('qasm_simulator')
    pm = PassManager()
    pm.append([Optimize1qGates(), CommutativeCancellation(), Collect2qBlocks(), UnitarySynthesis(),
               RemoveDiagonalGatesBeforeMeasure(), OptimizeSwapBeforeMeasure(), TemplateOptimization()])
    transpiled_qiskit = pm.run(qc)
    return  transpiled_qiskit

optimized_circuit = qiskit_transpiler(qc)
print(qc.draw())
print(optimized_circuit.draw())

     ┌──────────┐
q_0: ┤0         ├
     │  Unitary │
q_1: ┤1         ├
     └──────────┘
     ┌──────────┐
q_0: ┤0         ├
     │  Unitary │
q_1: ┤1         ├
     └──────────┘


In [43]:
qc = QuantumCircuit(2)
qc.cz(0,1)
qc.h(1)
qc.cx(0,1)
qc.h(0)
qc.cz(0,1)
qc.h(1)
optimized_qc = transpile(qc, basis_gates=['h', 'x', 'cx', 'cz', 'ry', 'u', 'u1', 'u2'], optimization_level=2) 
print(optimized_qc.draw())
optimized_qc = transpile(qc, basis_gates=['h', 'x', 'cx', 'cz', 'ry', 'u', 'u1', 'u2'], optimization_level=3) 
print(optimized_qc.draw())

# unitary synthesis -> transpile
qc = QuantumCircuit(2)
qc.unitary(unitary, range(2))
optimized_qc = transpile(qc, basis_gates=['h', 'x', 'cx', 'cz', 'ry', 'u', 'u1', 'u2'], optimization_level=2) 
print(optimized_qc.draw())
optimized_qc = transpile(qc, basis_gates=['h', 'x', 'cx', 'cz', 'ry', 'u', 'u1', 'u2'], optimization_level=3) 
print(optimized_qc.draw())

global phase: 4.0243
      ┌──────────────┐   ┌─────────────────────┐
q_0: ─┤ U(π/2,π/2,0) ├─■─┤ U(π,2.2589,-2.4535) ├
     ┌┴──────────────┤ │ └──┬───────────────┬──┘
q_1: ┤ U(π/2,π/2,-π) ├─■────┤ U(π/2,0,-π/2) ├───
     └───────────────┘      └───────────────┘   
global phase: 4.0243
      ┌──────────────┐   ┌─────────────────────┐
q_0: ─┤ U(π/2,π/2,0) ├─■─┤ U(π,2.2589,-2.4535) ├
     ┌┴──────────────┤ │ └──┬───────────────┬──┘
q_1: ┤ U(π/2,π/2,-π) ├─■────┤ U(π/2,0,-π/2) ├───
     └───────────────┘      └───────────────┘   
global phase: 0
     ┌───────────────────┐   ┌───────────────────┐
q_0: ┤ U(π/2,-0.32041,0) ├─■─┤ U(π/2,-π,-2.8212) ├
     └───────────────────┘ │ └─┬───────────────┬─┘
q_1: ──────────────────────■───┤ U(0,π/4,3π/4) ├──
                               └───────────────┘  
global phase: 0
     ┌───────────────────┐   ┌───────────────────┐
q_0: ┤ U(π/2,-0.32041,0) ├─■─┤ U(π/2,-π,-2.8212) ├
     └───────────────────┘ │ └─┬───────────────┬─┘
q_1: ──────────────────────■

In [ ]:
qc.count_ops().keys()